In [2]:
# Tratamiento de datos
# ==============================================================================
import pandas as pd
import numpy as np


# Almacenar en caché los resultados de funciones en el disco
# ==============================================================================
import joblib


# Gestion de librerias
# ==============================================================================
from importlib import reload


# Matemáticas y estadísticas
# ==============================================================================
import math


# Preprocesado y modelado
# ==============================================================================

#Separar los datos entrenamiento y prueba
from sklearn.model_selection import train_test_split


#Escalar Variables
from sklearn.preprocessing import MinMaxScaler


#Evaluación del modelo
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve


#Creación de modelo
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
#from sklearn.tree import export_graphviz
#from sklearn.tree import export_text


#configuracion de hiperparámetros
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns


# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

# Funciones externas
# ==============================================================================
from manny_library import multiple_plot, plot_roc_curve

In [3]:
df = pd.read_csv('df.csv')

## Transformaciones

In [4]:
# Aplicación de la función de usuario Dummies: one-hot encoding

df = pd.get_dummies(df, drop_first=True) #.astype(int)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5722 entries, 0 to 5721
Columns: 109 entries, NIVEL DE SUPERVISION to CATEGORIA_Fondo de Empleados
dtypes: bool(63), float64(44), int64(2)
memory usage: 2.4 MB


In [5]:
# Se establecen las variables numéricas a escalar

# Num_vars se le asigna la lista con las variables numericas para posteriormente escalarlas
num_vars = df.select_dtypes(include=['float64', 'int32', 'int64']).columns.to_list()

num_vars.remove('EXCEDENTES Y/O PÉRDIDAS  DEL EJERCICIO')

print(num_vars)

['NIVEL DE SUPERVISION', 'ASOCIADOS', 'EMPLEADOS', 'ACTIVO', 'EFECTIVO Y EQUIVALENTE AL EFECTIVO', 'INVERSIONES', 'INVENTARIOS', 'CARTERA DE CRÉDITOS', 'CUENTAS POR COBRAR Y OTRAS', 'ACTIVOS MATERIALES', 'ACTIVOS NO CORRIENTES MANTENIDOS PARA LA VENTA', 'OTROS ACTIVOS', 'PASIVOS', 'DEPOSITOS', 'OBLIGACIONES FINANCIERAS Y OTROS PASIVOS FINANCIEROS', 'CUENTAS POR PAGAR Y OTRAS', 'IMPUESTO DIFERIDO PASIVO', 'FONDOS SOCIALES Y MUTUALES', 'OTROS PASIVOS', 'PROVISIONES', 'PATRIMONIO', 'CAPITAL SOCIAL', 'APORTES SOCIALES TEMPORALMENTE RESTRINGIDOS', 'RESERVAS', 'FONDOS  DE DESTINACIÓN ESPECÍFICA', 'SUPERÁVIT', 'RESULTADOS ACUMULADOS POR ADOPCIÓN POR PRIMERA VEZ', 'INGRESOS', 'INGRESOS POR VENTA DE BIENES Y SERVICIOS', 'OTROS INGRESOS', 'GASTOS', 'GASTOS DE ADMINISTRACION', 'OTROS GASTOS', 'GASTOS DE VENTAS', 'COSTO DE VENTAS', 'COSTO DE VENTAS Y DE PRESTACION DE SERVICIOS', 'COMPRAS', 'DEUDORAS CONTINGENTES', 'DEUDORAS DE CONTROL', 'DEUDORAS CONTINGENTES POR CONTRA (CR)', 'DEUDORAS DE CONTROL

## Creacion del modelo

In [6]:
# Se establece las variables de entrada 'X' y la variable de salida 'y'
X = df.drop(columns = 'EXCEDENTES Y/O PÉRDIDAS  DEL EJERCICIO')
y = df['EXCEDENTES Y/O PÉRDIDAS  DEL EJERCICIO']

In [7]:
# Se crean los datasets de entrenamiento y prueba para las variables de entrada y salida

X_train, X_test, y_train, y_test = train_test_split(
                                        X,
                                        y.values.reshape(-1,1),
                                        train_size   = 0.8,
                                        random_state = 123,
                                        shuffle      = True
                                    )

#### Escalar variables

In [8]:
# Escalar Variables númericas

pd.set_option('display.float_format', lambda x: '%.4f' % x)

# Se crea un objeto MinMaxScaler
scaler = MinMaxScaler()

# Se escalan los valores del dataset entrenamiento y prueba de las columnas numéricas
X_train[num_vars] = scaler.fit_transform(X_train[num_vars])
X_test[num_vars] = scaler.transform(X_test[num_vars])

X_train[num_vars].head(2)

,NIVEL DE SUPERVISION,ASOCIADOS,EMPLEADOS,ACTIVO,EFECTIVO Y EQUIVALENTE AL EFECTIVO,INVERSIONES,INVENTARIOS,CARTERA DE CRÉDITOS,CUENTAS POR COBRAR Y OTRAS,ACTIVOS MATERIALES,...,COSTO DE VENTAS Y DE PRESTACION DE SERVICIOS,COMPRAS,DEUDORAS CONTINGENTES,DEUDORAS DE CONTROL,DEUDORAS CONTINGENTES POR CONTRA (CR),DEUDORAS DE CONTROL POR CONTRA (CR),ACREEDORAS CONTINGENTES,ACREEDORAS DE CONTROL,ACREEDORAS POR CONTRA (DB),ACREEDORAS DE CONTROL POR CONTRA (CR)
524,1.0000,0.0003,0.0002,0.0017,0.0054,0.0000,0.0000,0.0000,0.0044,0.0000,...,0.0041,0.0001,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
5259,1.0000,0.0001,0.0012,0.0005,0.0024,0.0001,0.0000,0.0000,0.0002,0.0004,...,0.0010,0.0001,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


#### Seleccion de variables por medio de Arboles de Decision

In [9]:
# Creación del modelo
# ------------------------------------------------------------------------------
modelDtree = DecisionTreeClassifier(
            max_depth         = 5,
            criterion         = 'gini',
            random_state      = 123
          )

# Entrenamiento del modelo
# ------------------------------------------------------------------------------
modelDtree.fit(X_train, y_train)

,criterion,'gini'
,splitter,'best'
,max_depth,5
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,123
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


In [10]:
print("Importancia de los predictores en el modelo")
print("-------------------------------------------")
importancia_predictores = pd.DataFrame(
                            {'predictor': X.columns,
                             'importancia': modelDtree.feature_importances_}
                            )
importancia_predictores.sort_values('importancia', ascending=False)

Importancia de los predictores en el modelo
-------------------------------------------


,predictor,importancia
7,CARTERA DE CRÉDITOS,0.3948
4,EFECTIVO Y EQUIVALENTE AL EFECTIVO,0.0786
37,DEUDORAS CONTINGENTES,0.0751
30,GASTOS,0.0730
9,ACTIVOS MATERIALES,0.0680
...,...,...
103,DEPARTAMENTO_VALLE DEL CAUCA,0.0000
104,DEPARTAMENTO_VAUPÉS,0.0000
105,CATEGORIA_CACs,0.0000
106,CATEGORIA_CDACs,0.0000


In [21]:
final_columns = importancia_predictores[importancia_predictores.importancia > 0.025]
final_columns = final_columns['predictor'].tolist()
final_columns

['ASOCIADOS',
 'EFECTIVO Y EQUIVALENTE AL EFECTIVO',
 'CARTERA DE CRÉDITOS',
 'ACTIVOS MATERIALES',
 'CUENTAS POR PAGAR Y OTRAS',
 'PATRIMONIO',
 'CAPITAL SOCIAL',
 'RESERVAS',
 'GASTOS',
 'DEUDORAS CONTINGENTES',
 'DEUDORAS DE CONTROL',
 'DEUDORAS DE CONTROL POR CONTRA (CR)']

In [25]:
final_dataset = pd.concat([df[final_columns], df['EXCEDENTES Y/O PÉRDIDAS  DEL EJERCICIO']], axis=1) 
final_dataset

,ASOCIADOS,EFECTIVO Y EQUIVALENTE AL EFECTIVO,CARTERA DE CRÉDITOS,ACTIVOS MATERIALES,CUENTAS POR PAGAR Y OTRAS,PATRIMONIO,CAPITAL SOCIAL,RESERVAS,GASTOS,DEUDORAS CONTINGENTES,DEUDORAS DE CONTROL,DEUDORAS DE CONTROL POR CONTRA (CR),EXCEDENTES Y/O PÉRDIDAS DEL EJERCICIO
0,437.0000,600768784.6500,1520753496.0000,438279168.0000,36652884.4400,2483719702.2100,1562957617.6400,567021979.7600,328039145.6600,32447957.0000,2311.6600,2311.6600,0
1,451.0000,1378691393.3600,6789256269.0000,766661.0000,160666756.0000,2006480350.0000,575489867.0000,1380294375.2900,769426187.1700,15847179.0000,0.0000,0.0000,1
2,996.0000,4584834357.9700,50659704563.6900,13034321.0000,430827404.9800,38167917256.3900,33924122287.1100,1819618428.1200,5439064134.7500,1779198.0000,64387139.6900,51825623278.3800,1
3,640.0000,338788013.0800,1529101057.0000,364358410.0000,22872424.0000,2350450558.5000,1424958424.0000,618545162.9000,318836979.7800,0.0000,0.0000,0.0000,0
4,1022.0000,828278362.2800,6498208882.9600,1262487424.9900,49994728.0500,8592656217.0400,7631953020.2200,331252006.9000,952126997.0900,44259233.0000,174814163.0000,174814163.0000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5717,102.0000,62437850.8600,402868074.0000,285648.0000,10013244.0000,510937106.3600,345286183.0000,146710402.7200,188609766.1700,0.0000,6221260.0000,6221260.0000,0
5718,185.0000,10572989.0000,0.0000,0.0000,4217371.0000,3817283.0000,55000000.0000,0.0000,36428485.0000,0.0000,0.0000,0.0000,0
5719,46.0000,46587992.0000,266304078.0000,0.0000,2709600.0000,55407571.9200,53278412.0000,2734492.9200,15991855.0000,0.0000,0.0000,0.0000,0
5720,23.0000,3176539.6700,34469795.0000,0.0000,0.0000,37926576.1900,41929233.0000,2470991.2400,4789040.0000,0.0000,0.0000,0.0000,0


In [26]:
final_dataset.to_csv('final_dataset.csv', index=False)